In [34]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from isometry_pursuit.plot import box_plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# power = 1.
# R = 25
# frac = .1
# D_iris = 4
# name = "iris_standardized"
# filename = f'/Users/samsonkoelle/isometry-pursuit/experiments/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}'
# results_iris_p1 = pd.read_csv(filename)

# box_plot(results_iris_p1['Brute Isometry Losses'],
#          results_iris_p1['Two-Stage Losses'],
#          D_iris,
#          xlabels = [r'$\widehat S_{B(\cdot,l_1)}$', r'$\widehat S_{TSIP}$'],
#          ylabel = r'$l_1(X_{.\widehat {S}}) - D$',
#          filename = f'/Users/samsonkoelle/isometry-pursuit/figures/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}_isometry_losses')

# box_plot(results_iris_p1['Brute Losses'],
#          results_iris_p1['Greedy Multitask Norms (Two-Stage)'],
#          D_iris,
#          xlabels = [r'$\widehat S_{B(\cdot, {\|\|_{1,2}})}$', r'$\widehat S_{B( \cdot,{\|\|_{1,2}}) \circ IP}$'],
#          ylabel = r'$\|(w(X_{.\widehat {S}}, 1))\|_{1,2} - D$',
#          filename = f'/Users/samsonkoelle/isometry-pursuit/figures/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}_group_lasso_losses')

# power = 1.
# R = 25
# frac = .1
# name = "wine_standardized"
# filename = f'/Users/samsonkoelle/isometry-pursuit/experiments/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}'
# results_wine_p1 = pd.read_csv(filename)
# D_wine = 13

# box_plot(results_wine_p1['Brute Isometry Losses'],
#          results_wine_p1['Two-Stage Losses'],
#          D_wine,
#          xlabels = [r'$\widehat S_{B(\cdot,l_1)}$', r'$\widehat S_{TSIP}$'],
#          ylabel = r'$l_1(X_{.\widehat {S}}) - D$',
#          filename = f'/Users/samsonkoelle/isometry-pursuit/figures/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}_isometry_losses')

# box_plot(results_wine_p1['Brute Losses'],
#          results_wine_p1['Greedy Multitask Norms (Two-Stage)'],
#          D_wine,
#          xlabels = [r'$\widehat S_{B(\cdot, {\|\|_{1,2}})}$', r'$\widehat S_{B( \cdot,{\|\|_{1,2}}) \circ IP}$'],
#          ylabel = r'$\|w((X_{.\widehat {S}}, 1))\|_{1,2} - D$',
#          filename = f'/Users/samsonkoelle/isometry-pursuit/figures/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}_group_lasso_losses')

In [41]:
power = 1.
D_wine = 6
frac = .5
R = 25
name = "wine_standardized"
filename = f'/Users/samsonkoelle/isometry-pursuit/experiments/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}'
results_wine_p5 = pd.read_csv(filename)
box_plot(results_wine_p5['Losses'],
         results_wine_p5['Two-Stage Losses'],
         D_iris,
         xlabels = [r'$\widehat S_{g}$', r'$\widehat S_{TSIP}$'],
         ylabel = r'$l_1(X_{.\widehat {S}}) - D$',
         filename = f'/Users/samsonkoelle/isometry-pursuit/figures/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}_isometry_losses')

name = "iris_standardized"
filename = f'/Users/samsonkoelle/isometry-pursuit/experiments/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}'
results_iris_p5 = pd.read_csv(filename)
box_plot(results_iris_p5['Losses'],
         results_iris_p5['Two-Stage Losses'],
         D_iris,
         xlabels = [r'$\widehat S_{g}$', r'$\widehat S_{TSIP}$'],
         ylabel = r'$l_1(X_{.\widehat {S}}) - D$',
         filename = f'/Users/samsonkoelle/isometry-pursuit/figures/{name}_{str(frac).replace(".","p")}_{str(power).replace(".","p")}_isometry_losses')


power = 1.
R = 100
frac = .1
filename = f'/Users/samsonkoelle/isometry-pursuit/experiments/ethanol_results.csv'
results_ethanol = pd.read_csv(filename)
D_ethanol = 2

box_plot(results_ethanol['Losses'],
         results_ethanol['Two-Stage Losses'],
         D_ethanol,
         xlabels = [r'$\widehat S_{g}$', r'$\widehat S_{TSIP}$'],
         ylabel = r'$l_1(X_{.\widehat {S}}) - D$',
         filename = f'/Users/samsonkoelle/isometry-pursuit/figures/ethanol_isometry_losses')

# Wine
from isometry_pursuit.plot import ecdf_plot
ecdf_plot(
    results_wine_p5['Losses'],
    results_wine_p5['Two-Stage Losses'],
    "Wine",
    xlabel = r'$\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP})$',
    ylabel = r'$\widehat{P}(\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP}) \leq t)$',
    filename = f'/Users/samsonkoelle/isometry-pursuit/figures/wine_standardized_isometry_losses_ecdf'
)

# Iris
ecdf_plot(
    results_iris_p5['Losses'],
    results_iris_p5['Two-Stage Losses'],
    "Iris",
    xlabel = r'$\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP})$',
    ylabel = r'$\widehat{P}(\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP}) \leq t)$',
    filename = f'/Users/samsonkoelle/isometry-pursuit/figures/iris_standardized_isometry_losses_ecdf'
)

# Ethanol
ecdf_plot(
    results_ethanol['Losses'],
    results_ethanol['Two-Stage Losses'],
    'Ethanol tangent',
    xlabel = r'$\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP})$',
    ylabel = r'$\widehat{P}(\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP}) \leq t)$',
    filename = '/Users/samsonkoelle/isometry-pursuit/figures/ethanol_isometry_losses_ecdf'
)

name = "words_standardized"
filename = f'/Users/samsonkoelle/isometry-pursuit/experiments/words'
results_words  = pd.read_csv(filename)
# Ethanol
ecdf_plot(
    results_words['Losses'],
    results_words['Two-Stage Losses'],
    "Words",
    xlabel = r'$\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP})$',
    ylabel = r'$\widehat{P}(\ell_1(\widehat{S}_g) - \ell_1(\widehat{S}_{TSIP}) \leq t)$',
    filename = '/Users/samsonkoelle/isometry-pursuit/figures/words_ecdf'
)

In [69]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel

# Helper for 2 significant figures
def sigfig(x, n=2):
    return f"{x:.{n}g}"

# Compute p-values using paired two-sample t-test
iris_p_value = ttest_rel(results_iris_p5['Losses'], results_iris_p5['Two-Stage Losses']).pvalue
wine_p_value = ttest_rel(results_wine_p5['Losses'], results_wine_p5['Two-Stage Losses']).pvalue
ethanol_p_value = ttest_rel(results_ethanol['Losses'], results_ethanol['Two-Stage Losses']).pvalue
words_p_value = ttest_rel(results_words['Losses'], results_words['Two-Stage Losses']).pvalue

# Data summarization for the table
data = {
    "Name": ["Iris", "Wine", "Ethanol"],
    "$D$": [4, 6, 2],
    "$P$": [75, 89, 756],
    "$R$": [25, 25, 100],
    "$c$": [1, 1, 1],
    "$\ell_1(X_{.\\widehat{S}_{G}})$": [
        f"{sigfig(results_iris_p5['Losses'].mean())} ± {sigfig(results_iris_p5['Losses'].std())}",
        f"{sigfig(results_wine_p5['Losses'].mean())} ± {sigfig(results_wine_p5['Losses'].std())}",
        f"{sigfig(results_ethanol['Losses'].mean())} ± {sigfig(results_ethanol['Losses'].std())}"
    ],
    "$|\\widehat{S}_{TSIP}|$": [
        f"{sigfig(results_iris_p5['Support Cardinalities (Basis Pursuit)'].mean())} ± {sigfig(results_iris_p5['Support Cardinalities (Basis Pursuit)'].std())}",
        f"{sigfig(results_wine_p5['Support Cardinalities (Basis Pursuit)'].mean())} ± {sigfig(results_wine_p5['Support Cardinalities (Basis Pursuit)'].std())}",
        f"{sigfig(results_ethanol['Support Cardinalities (Basis Pursuit)'].mean())} ± {sigfig(results_ethanol['Support Cardinalities (Basis Pursuit)'].std())}"
    ],
    "$\ell_1(X_{.\\widehat{S}})$": [
        f"{sigfig(results_iris_p5['Two-Stage Losses'].mean())} ± {sigfig(results_iris_p5['Two-Stage Losses'].std())}",
        f"{sigfig(results_wine_p5['Two-Stage Losses'].mean())} ± {sigfig(results_wine_p5['Two-Stage Losses'].std())}",
        f"{sigfig(results_ethanol['Two-Stage Losses'].mean())} ± {sigfig(results_ethanol['Two-Stage Losses'].std())}"
    ],
    "$P_R (\ell_1(X_{.\\widehat{S}_{G}}) > \ell_1(X_{.\\widehat{S}_{TSIP}}))$": [
        sigfig((results_iris_p5['Losses'] > results_iris_p5['Two-Stage Losses']).mean()),
        sigfig((results_wine_p5['Losses'] > results_wine_p5['Two-Stage Losses']).mean()),
        sigfig((results_ethanol['Losses'] > results_ethanol['Two-Stage Losses']).mean())
    ],
    "$P_R (\ell_1(X_{.\\widehat{S}_{G}}) = \ell_1(X_{.\\widehat{S}_{TSIP}}))$": [
        sigfig((results_iris_p5['Losses'] == results_iris_p5['Two-Stage Losses']).mean()),
        sigfig((results_wine_p5['Losses'] == results_wine_p5['Two-Stage Losses']).mean()),
        sigfig((results_ethanol['Losses'] == results_ethanol['Two-Stage Losses']).mean())
    ],
    "$\widehat P(\\bar{\ell}_1(X_{.\\widehat{S}_{G}}) > \\bar{\ell}_1(X_{.\\widehat{S}_{TSIP}}))$": [
        f"{iris_p_value:.1e}",
        f"{wine_p_value:.1e}",
        f"{ethanol_p_value:.1e}"
    ]
}

# Add 'Words' row
data["Name"].append("Words")
data["$D$"].append(6)
data["$P$"].append(61)
data["$R$"].append(25)
data["$c$"].append(1)
data["$\ell_1(X_{.\\widehat{S}_{G}})$"].append(
    f"{sigfig(results_words['Losses'].mean())} ± {sigfig(results_words['Losses'].std())}"
)
data["$|\\widehat{S}_{TSIP}|$"].append(
    f"{sigfig(results_words['Support Cardinalities (Basis Pursuit)'].mean())} ± {sigfig(results_words['Support Cardinalities (Basis Pursuit)'].std())}"
)
data["$\ell_1(X_{.\\widehat{S}})$"].append(
    f"{sigfig(results_words['Two-Stage Losses'].mean())} ± {sigfig(results_words['Two-Stage Losses'].std())}"
)
data["$P_R (\ell_1(X_{.\\widehat{S}_{G}}) > \ell_1(X_{.\\widehat{S}_{TSIP}}))$"].append(
    sigfig((results_words['Losses'] > results_words['Two-Stage Losses']).mean())
)
data["$P_R (\ell_1(X_{.\\widehat{S}_{G}}) = \ell_1(X_{.\\widehat{S}_{TSIP}}))$"].append(
    sigfig((results_words['Losses'] == results_words['Two-Stage Losses']).mean())
)
data["$\widehat P(\\bar{\ell}_1(X_{.\\widehat{S}_{G}}) > \\bar{\ell}_1(X_{.\\widehat{S}_{TSIP}}))$"].append(
    f"{words_p_value:.1e}"
)

# Create DataFrame and export LaTeX
df_table = pd.DataFrame(data)
latex_table = df_table.to_latex(
    index=False,
    escape=False,
    column_format="|c|c|c|c|c|c|c|c|c|c|c|",
    header=True
)

print(latex_table)


\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|}
\toprule
Name & $D$ & $P$ & $R$ & $c$ & $\ell_1(X_{.\widehat{S}_{G}})$ & $|\widehat{S}_{TSIP}|$ & $\ell_1(X_{.\widehat{S}})$ & $P_R (\ell_1(X_{.\widehat{S}_{G}}) > \ell_1(X_{.\widehat{S}_{TSIP}}))$ & $P_R (\ell_1(X_{.\widehat{S}_{G}}) = \ell_1(X_{.\widehat{S}_{TSIP}}))$ & $\widehat P(\bar{\ell}_1(X_{.\widehat{S}_{G}}) > \bar{\ell}_1(X_{.\widehat{S}_{TSIP}}))$ \\
\midrule
Iris & 4 & 75 & 25 & 1 & 14 ± 7.3 & 6.7 ± 1 & 6.9 ± 1.4 & 0.96 & 0 & 2.4e-05 \\
Wine & 6 & 89 & 25 & 1 & 7.7 ± 0.33 & 13 ± 1.5 & 7.6 ± 0.29 & 0.64 & 0.16 & 6.3e-04 \\
Ethanol & 2 & 756 & 100 & 1 & 2.6 ± 0.3 & 90 ± 1.7e+02 & 2.5 ± 0.2 & 0.66 & 0.17 & 2.1e-05 \\
Words & 6 & 61 & 25 & 1 & 14 ± 1.3 & 11 ± 1.3 & 14 ± 1.2 & 0.52 & 0.12 & 2.1e-02 \\
\bottomrule
\end{tabular}



In [71]:
data = {
    "Name": ["Iris", "Wine", "Ethanol", "Words"],
    "IP": [
        f"{results_iris_p5['Basis Pursuit Times'].mean():.2f} ± {results_iris_p5['Basis Pursuit Times'].std():.2f}",
        f"{results_wine_p5['Basis Pursuit Times'].mean():.2f} ± {results_wine_p5['Basis Pursuit Times'].std():.2f}",
        f"{results_ethanol['Basis Pursuit Times'].mean():.2f} ± {results_ethanol['Basis Pursuit Times'].std():.2f}",
        f"{results_words['Basis Pursuit Times'].mean():.2f} ± {results_words['Basis Pursuit Times'].std():.2f}",
    ],
    "2nd stage": [
        f"{results_iris_p5['Stage Two Times'].mean():.2f} ± {results_iris_p5['Stage Two Times'].std():.2f}",
        f"{results_wine_p5['Stage Two Times'].mean():.2f} ± {results_wine_p5['Stage Two Times'].std():.2f}",
        f"{results_ethanol['Stage Two Times'].mean():.2f} ± {results_ethanol['Stage Two Times'].std():.2f}",
        f"{results_words['Stage Two Times'].mean():.2f} ± {results_words['Stage Two Times'].std():.2f}",
    ],
    "Greedy": [
        f"{results_iris_p5['Greedy Times'].mean():.2f} ± {results_iris_p5['Greedy Times'].std():.2f}",
        f"{results_wine_p5['Greedy Times'].mean():.2f} ± {results_wine_p5['Greedy Times'].std():.2f}",
        f"{results_ethanol['Greedy Times'].mean():.2f} ± {results_ethanol['Greedy Times'].std():.2f}",
        f"{results_words['Greedy Times'].mean():.2f} ± {results_words['Greedy Times'].std():.2f}",
    ],
}


In [72]:
# Create a DataFrame
df_table = pd.DataFrame(data).iloc[:,]
# Convert to LaTeX
latex_table = df_table.to_latex(
    index=False,
    escape=False,  # Prevent escaping special characters
    column_format="|c|c|c|c",  # Column format matches your table
    header=True
)

print(latex_table)

\begin{tabular}{|c|c|c|c}
\toprule
Name & IP & 2nd stage & Greedy \\
\midrule
Iris & 1.24 ± 0.02 & 0.00 ± 0.00 & 0.02 ± 0.00 \\
Wine & 2.32 ± 0.17 & 0.13 ± 0.12 & 0.03 ± 0.00 \\
Ethanol & 8.38 ± 0.57 & 0.55 ± 1.08 & 0.07 ± 0.01 \\
Words & 0.98 ± 0.20 & 0.03 ± 0.02 & 0.01 ± 0.00 \\
\bottomrule
\end{tabular}



In [76]:
import matplotlib.pyplot as plt

# Define Garfield High School Purple color
garfield_purple = "#4E2A84"

# --- Wine dataset ---
plt.figure()
plt.hist(
    results_wine_p5['Support Cardinalities (Basis Pursuit)'],
    bins=range(
        min(results_wine_p5['Support Cardinalities (Basis Pursuit)']),
        max(results_wine_p5['Support Cardinalities (Basis Pursuit)']) + 2
    ),
    color=garfield_purple, edgecolor='white', align='left'
)
plt.xlabel(r'$|\widehat{S_{IP}}|$', fontsize=30)
plt.ylabel('Frequency', fontsize=24)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('/Users/samsonkoelle/isometry-pursuit/figures/wine_cardinalities')
plt.close()

# --- Iris dataset ---
plt.figure()
plt.hist(
    results_iris_p5['Support Cardinalities (Basis Pursuit)'],
    bins=range(
        min(results_iris_p5['Support Cardinalities (Basis Pursuit)']),
        max(results_iris_p5['Support Cardinalities (Basis Pursuit)']) + 2
    ),
    color=garfield_purple, edgecolor='white', align='left'
)
plt.xlabel(r'$|\widehat{S_{IP}}|$', fontsize=30)
plt.ylabel('Frequency', fontsize=24)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('/Users/samsonkoelle/isometry-pursuit/figures/iris_cardinalities')
plt.close()

# --- Ethanol dataset (25-wide bins) ---
plt.figure()
ethanol_min = min(results_ethanol['Support Cardinalities (Basis Pursuit)'])
ethanol_max = max(results_ethanol['Support Cardinalities (Basis Pursuit)'])
plt.hist(
    results_ethanol['Support Cardinalities (Basis Pursuit)'],
    bins=range(ethanol_min, ethanol_max + 26, 25),
    color=garfield_purple, edgecolor='white'
)
plt.xlabel(r'$|\widehat{S_{IP}}|$', fontsize=30)
plt.ylabel('Frequency', fontsize=24)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('/Users/samsonkoelle/isometry-pursuit/figures/ethanol_cardinalities')
plt.close()

# --- Words dataset ---
plt.figure()
words_min = min(results_words['Support Cardinalities (Basis Pursuit)'])
words_max = max(results_words['Support Cardinalities (Basis Pursuit)'])
plt.hist(
    results_words['Support Cardinalities (Basis Pursuit)'],
    bins=range(words_min, words_max + 2),
    color=garfield_purple, edgecolor='white', align='left'
)
plt.xlabel(r'$|\widehat{S_{IP}}|$', fontsize=30)
plt.ylabel('Frequency', fontsize=24)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('/Users/samsonkoelle/isometry-pursuit/figures/words_cardinalities')
plt.close()
